In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

### Tables in the process

In [2]:
cols = 'trade name qty_x price reason_x qty_y buy_target sell_target reason_y'.split()
colt = 'trade name qty price active reason market'.split()

In [3]:
format_dict = {
    'qty':'{:,}','price':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGC,"10,000",10.40,"104,000.00",DOS,SET,2
1,B,GLOBAL,"7,500",19.00,"142,500.00",RD05%,SET50,1
2,B,KCE,"2,000",82.50,"165,000.00",RD05%,SET50,2
3,B,NOBLE,"21,000",5.85,"122,850.00",RD15%,SET,1
4,B,PTT,"3,000",36.50,"109,500.00",RD05%,SET50,1
5,B,RATCH,"3,000",42.75,"128,250.00",RD05%,SET50,1
6,B,ROJNA,"18,000",6.50,"117,000.00",6L,SET,2
7,B,SIS,"3,000",42.00,"126,000.00",DFM10%,SET,0
8,B,STA,"2,500",28.00,"70,000.00",RD15%,SET50,0
9,B,SYNEX,"6,000",31.50,"189,000.00",DFM05%,SET100,2


In [5]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlite)
stocks.shape

(52, 18)

In [6]:
df_merge = pd.merge(orders,stocks,on='name',how='inner')
df_merge.shape

(22, 25)

In [7]:
qty_mask = (df_merge.qty_x != df_merge.qty_y)
df_merge[qty_mask][cols]

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y


In [ ]:
name = 'KCE'
sqlUpd = """
UPDATE stocks
SET qty = 2000
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount

In [12]:
buy_price_mask = (df_merge.price != df_merge.buy_target) & (df_merge.trade == 'B')
df_merge[buy_price_mask][cols]

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y


In [ ]:
name = 'BCH'
sqlUpd = """
UPDATE stocks
SET buy_target = 21.3, sell_target = 23.4
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount

In [13]:
sell_price_mask = (df_merge.price != df_merge.sell_target) & (df_merge.trade == 'S')
df_merge[sell_price_mask][cols]

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y


In [ ]:
name = 'BCH'
sqlUpd = """
UPDATE stocks
SET sell_target = 23.4
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount

In [14]:
reason_mask = (df_merge.reason_x != df_merge.reason_y)
df_merge[reason_mask][cols]

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y
2,B,KCE,2000,82.50,RD05%,2000,82.5,0.00,5pct
9,B,SYNEX,6000,31.50,DFM05%,6000,31.5,0.00,10%
17,S,KBANK,800,150.00,10%,800,0.0,150.00,10pct
19,S,RJH,1500,36.25,10%,1500,0.0,36.25,10pct
20,S,SAT,2500,24.25,15%,2500,0.0,24.25,15pct


In [15]:
name = 'SAT'
sqlUpd = """
UPDATE stocks
SET reason = '15pct'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET reason = '15pct'
WHERE name = 'SAT'



1

### End of Loop process

In [16]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

orders[colt].to_csv(output_file, header=True, index=False)
orders[colt].to_csv(data_file,   header=True, index=False)
orders[colt].to_csv(box_file,    header=True, index=False)

### After call ord-log

In [17]:
def categorise(row):  
    if row['col0'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [18]:
df = pd.read_csv('../data/orders-log.csv', header=None, prefix = 'col',usecols=[0,1,2,3,4,5,6,7,8,9])
df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market','col5': 'qty', 'col6': 'target','col7': 'current', 'col8': 'chg', 'col9': 'percent'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

In [19]:
from itables import show
show(df[['trans','name','spd','reason','qty','target','current','chg','percent']])

<IPython.core.display.Javascript object>

trans,name,spd,reason,qty,target,current,chg,percent


In [20]:
sql = """
UPDATE stocks SET buy_target=(
SELECT orders.price 
FROM orders 
WHERE orders.name=stocks.name AND orders.trade='B')"""
rp = conlite.execute(sql)
rp.rowcount

52

In [21]:
sql = '''
UPDATE stocks
SET buy_target = 0
WHERE buy_target IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

38

In [22]:
sql = '''
UPDATE stocks SET sell_target=(
SELECT orders.price 
FROM orders 
WHERE orders.name=stocks.name AND orders.trade='S')'''
rp = conlite.execute(sql)
rp.rowcount

52

In [23]:
sql = '''
UPDATE stocks
SET sell_target = 0
WHERE sell_target IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

44

In [24]:
sql = '''
UPDATE stocks
SET available_qty = 0
WHERE available_qty IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [25]:
sql = '''
UPDATE stocks
SET cost = 0
WHERE cost IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [26]:
sql = '''
UPDATE stocks
SET qty = 0
WHERE qty IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [27]:
qty_mask = (stocks.qty == 0) & (stocks.status.isin(['B','I','O','S']))
stocks[qty_mask]

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market


In [28]:
buy_mask = (stocks.buy_target == 0) & (stocks.status.isin(['B','O']))
stocks[buy_mask]

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
27,631,EPG,13.7,7.7,B,0.0,0.0,414.49,0.43,11.2,18000,-9,12,14000,0,24H,RD05%,SET


In [29]:
name = 'EPG'
sqlUpd = """
UPDATE stocks
SET buy_target = 10.2
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET buy_target = 10.2
WHERE name = 'EPG'



1

In [30]:
sell_mask = (stocks.sell_target == 0) & (stocks.status.isin(['I','S']))
stocks[sell_mask]

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
2,280,IVL,50.00,33.75,I,0.0,0.0,1695.20,1.52,42.0,3000,-16,16,15000,1,12,10%,SET50
20,603,SYNEX,36.50,14.90,I,31.5,0.0,174.54,1.73,31.5,6000,-10,15,6000,1,12,10%,SET100
22,612,DOHOME,30.75,12.46,I,0.0,0.0,164.15,0.95,24.3,4000,-9,16,12000,1,0,10%,SET100
49,687,WHART,14.08,10.68,I,0.0,0.0,17.87,0.41,12.5,9000,-4,4,10000,0,0,DOS,SET


In [34]:
name = 'WHART'
sqlUpd = """
UPDATE stocks
SET sell_target = 12.8
WHERE name = '%s'
"""
sqlUpd = sqlUpd % name
print(sqlUpd)
rp = conlite.execute(sqlUpd)
rp.rowcount


UPDATE stocks
SET sell_target = 12.8
WHERE name = 'WHART'



1